# Homework 7

## ML 410


In the a previous homework assignments, you used two different dynamic programming algorithm to solve a robot navigation problem by finding optimal paths to a goal in a simplified warehouse environment. Now you will use classical reinforcement learning algorithms to find optimal paths in the same environment.

>**Note:** This assignment involves modifying code from several lesson notebooks. Please ask any questions about code on the course Piazza forum. 

Execute the code in the cell below to load the packages you will need for this exercise. 

In [1]:
## import required packages
import numpy as np
import numpy.random as nr
import itertools
import pandas as pd
from copy import deepcopy

np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 180

The configuration of the warehouse environment is illustrated in the figure below.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for Factory Navigation Example** </center>

The goal is for the robot to deliver some material to position (state) 12, shown in blue. Since there is a goal state or **terminal state** this an **episodic task**. 

There are some barriers comprised of the states $\{ 6, 7, 8 \}$ and $\{ 16, 17, 18 \}$, shown with hash marks. In a real warehouse, these positions might be occupied by shelving or equipment. We do not want the robot to hit these barriers. Thus, we say that transitioning to these barrier states is **taboo**.

As before, we do not want the robot to hit the edges of the grid world, which represent the outer walls of the warehouse. 

## Representation

You are, no doubt, familiar with the representation for this problem by now.    

As with many such problems, the starting place is creating the **representation**. In the cell below encode your representation for the possible action-state transitions. From each state there are 4 possible actions:
- up, u
- down, d,
- left, l
- right, r

There are a few special cases you need to consider:
- Any action transitioning state off the grid or into a barrier (taboo) state should keep the state unchanged. 
- Any action in the goal state keeps the state unchanged. 
- Any transition within the taboo (barrier) states can keep the state unchanged. If you experiment, you will see that other encodings work as well since the value of a barrier states are always zero and there are no actions transitioning into these states. 

In the cell below code the **dictionary  of dictionaries** which defines the possible state transitions to the neighboring states.

> **Hint:** It may help you create a pencil and paper sketch of the transitions, rewards, and probabilities or policy. This can help you to keep the bookkeeping correct. 

In [2]:
neighbors = {
    0: { 'u': 0, 'd': 5, 'l': 0, 'r': 1},
    1: { 'u': 1, 'd': 1, 'l': 0, 'r': 2},
    2: { 'u': 2, 'd': 2, 'l': 1, 'r': 3},
    3: { 'u': 3, 'd': 3, 'l': 2, 'r': 4},
    4: { 'u': 4, 'd': 9, 'l': 3, 'r': 4},
   
    5: { 'u': 0, 'd': 10, 'l': 5, 'r': 5},
    6: { 'u': 1, 'd': 11, 'l': 5, 'r': 7},   
    7: { 'u': 2, 'd': 12, 'l': 6, 'r': 8},
    8: { 'u': 3, 'd': 13, 'l': 7, 'r': 9},
    9: { 'u': 4, 'd': 14, 'l': 9, 'r': 9},
    
    10: { 'u': 5, 'd': 15, 'l': 10, 'r': 11},
    11: { 'u': 11, 'd': 11, 'l': 10, 'r': 12},
    12: { 'u': 12, 'd': 12, 'l': 12, 'r': 12},
    13: { 'u': 13, 'd': 13, 'l': 12, 'r': 14},
    14: { 'u': 9, 'd': 19, 'l': 13, 'r': 14},
    
    15: { 'u': 10, 'd': 20, 'l': 15, 'r': 15},
    16: { 'u': 11, 'd': 21, 'l': 15, 'r': 17},
    17: { 'u': 12, 'd': 22, 'l': 16, 'r': 18},
    18: { 'u': 13, 'd': 23, 'l': 17, 'r': 19},
    19: { 'u': 14, 'd': 24, 'l': 19, 'r': 19},
    
    20: { 'u': 15, 'd': 20, 'l': 20, 'r': 21},
    21: { 'u': 21, 'd': 21, 'l': 20, 'r': 22},
    22: { 'u': 22, 'd': 22, 'l': 21, 'r': 23},
    23: { 'u': 23, 'd': 23, 'l': 22, 'r': 24},
    24: { 'u': 19, 'd': 24, 'l': 23, 'r': 24}
}

The robot receives the following rewards:
- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the MC RL agent**. The agent must **learn** the rewards by sampling the environment. 

In the code cell below encode the **dictionary  of dictionaries** for your representation of this reward structure you will use in your simulated environment.  

In [3]:
rewards = {
    0: { 'u': -1.0, 'd': -0.1, 'l': -1.0, 'r': -0.1 },  
    1: { 'u': -1.0, 'd': -1.0, 'l': -0.1, 'r': -0.1 },
    2: { 'u': -1.0, 'd': -1.0, 'l': -0.1, 'r': -0.1 },
    3: { 'u': -1.0, 'd': -1.0, 'l': -0.1, 'r': -0.1 },
    4: { 'u': -1.0, 'd': -0.1, 'l': -0.1, 'r': -1.0 },

    5: { 'u': -0.1, 'd': -0.1, 'l': -1.0, 'r': -1.0 },
    6: { 'u': 0, 'd': 0, 'l': 0, 'r': 0 },
    7: { 'u': 0, 'd': 0, 'l': 0, 'r': 0 },
    8: { 'u': 0, 'd': 0, 'l': 0, 'r': 0 },
    9: { 'u': -0.1, 'd': -0.1, 'l': -1.0, 'r': -1.0 },
    
    10: { 'u': -0.1, 'd': -0.1, 'l': -1.0, 'r': -0.1 },
    11: { 'u': -1.0, 'd': -1.0, 'l': -0.1, 'r': 10 },
    12: { 'u': 0.0, 'd': 0.0, 'l': 0.0, 'r': 0.0 },
    13: { 'u': -1.0, 'd': -1.0, 'l': 10, 'r': -0.1 },
    14: { 'u': -0.1, 'd': -0.1, 'l': -0.1, 'r': -1.0 },
    
    15: { 'u': -0.1, 'd': -0.1, 'l': -1.0, 'r': -1.0 },
    16: { 'u': 0, 'd': 0, 'l': 0, 'r': 0 },
    17: { 'u': 0, 'd': 0, 'l': 0, 'r': 0 },
    18: { 'u': 0, 'd': 0, 'l': 0, 'r': 0 },
    19: { 'u': -0.1, 'd': -0.1, 'l': -1.0, 'r': -1.0 },
   
    20: { 'u': -0.1, 'd': -1.0, 'l': -1.0, 'r': -0.1 },
    21: { 'u': -1.0, 'd': -1.0, 'l': -0.1, 'r': -0.1 },
    22: { 'u': -1.0, 'd': -1.0, 'l': -0.1, 'r': -0.1 },
    23: { 'u': -1.0, 'd': -1.0, 'l': -0.1, 'r': -0.1 },
    24: { 'u': -0.1, 'd': -1.0, 'l': -0.1, 'r': -1.0 }
}    

### Environment Simulator

You must create a simulator for the grid world environment in the code below. Your RL agents will have no information on the environment, and will only interact through the simulator. The simulator must have at least the following characteristics:

1. Use the representations of the environment defined above.
2. Have arguments of state, action, terminal state and environment representation. 
3. Returns the state-prime, reward and if the state is the terminal state. 

You will also need a utility function to determine if a state is either terminal or taboo. This function will be used for generating random starts. 

In [4]:
terminal = 12
taboo = [6, 7, 8, 16, 17, 18]
def simulate(state, action):
    s_prime = neighbors[state][action]
    reward = rewards[state][action]
    return (s_prime, reward, is_terminal(s_prime), is_taboo(s_prime))

def is_terminal(state):
    return state == terminal

def is_taboo(state):
    return state in taboo

### Start Episode Function

To do random starts, you will need a function to generate them. Your function must ensure that the start state is not the terminal state or a taboo state. In the cell below create and test the required code.  

In [5]:
def start_episode(n_states):
    state = nr.choice(range(n_states))
    while(is_terminal(state) or is_taboo(state)):
         state = nr.choice(range(n_states))
    return state

### Initial Uniform Policy 

You need to define the initial transition probabilities for the Markov process. In the cell below create a **dictionary  of dictionaries** to set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. 

> **Note:** As these are just starting values, the exact values of the transition probabilities are not actually all that important in terms of solving the RL problem. Also, notice that it does not matter how the taboo state transitions are encoded. The point of the RL algorithm is to learn the transition policy.

In [6]:
initial_policy = {
    0: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    1: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    2: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    3: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    4: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    
    5: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    6: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    7: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    8: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    9: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    
    10: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    11: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    12: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
    13: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    14: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    
    15: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    16: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    17: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    18: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    19: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    
    20: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    21: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    22: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    23: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 },
    24: { 'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25 }
}

You will find it useful to create a list of taboo states, which you can encode in the cell below.

## Monte Carlo State Value Policy Evaluation

With your representations and the environment simulator defined, you can now create and test functions to perform MC RL **policy evaluation**. 

In the cell below create and test a function for taking an action in the environment. You may start with the `take_action` function from the lesson notebook. This function should have the following characteristics:

1. Have arguments of state and policy.
2. Compute the action, given the probabilities of the policy.
2. Return the action, the successor state, the reward, and a flag indicating if the successor state is terminal.  

In [7]:
def take_action(state, policy, actions = {1:'u', 2:'d', 3:'l', 4:'r'}):
    action = actions[nr.choice(range(len(actions)), p = list(policy[state].values())) + 1]
    s_prime, reward, is_terminal, is_taboo = simulate(state, action)
    if is_taboo:
        return take_action(state, policy, actions)
    return (action, s_prime, reward, is_terminal, is_taboo)

for s in range(16):
    print(take_action(s, initial_policy))

('r', 1, -0.1, False, False)
('d', 1, -1.0, False, False)
('l', 1, -0.1, False, False)
('u', 3, -1.0, False, False)
('r', 4, -1.0, False, False)
('u', 0, -0.1, False, False)
('u', 1, 0, False, False)
('u', 2, 0, False, False)
('r', 9, 0, False, False)
('r', 9, -1.0, False, False)
('r', 11, -0.1, False, False)
('u', 11, -1.0, False, False)
('l', 12, 0.0, True, False)
('l', 12, 10, True, False)
('u', 9, -0.1, False, False)
('d', 20, -0.1, False, False)


### Monte Carlo Episode Generation

As a first step you will need a function to generate a single first visit Monte Carlo episode for state value estimation, given the policy in the warehouse environment. You may start with the `MC_generate_episode` function from the MC RL notebook as a starting point.    

Make sure that taboo states are not visited on the random walk and that each episode ends at the terminal state. These key properties are 


In [8]:
def MC_episode(policy, G, n_visits, episode, n_states): 
    '''Function creates the Monte Carlo samples of one episode.
    This function does most of the real work'''
    ## For each episode we use a list to keep track of states we have visited.
    ## Once we visit a state we need to accumulate values to get the returns
    states_visited = []
    states = list(policy.keys())
        
    ## Find the starting state
    current_state = start_episode(n_states)
    terminal = False
    taboo = False
    g = 0.0
        
    while(not terminal or taboo):
        ## Find the next action and reward
        action, s_prime, reward, terminal, taboo = take_action(current_state, policy)
            
        ## Add the reward to the states visited if this is a first visit  
        if(current_state not in states_visited):
            ## Mark that the current state has been visited 
            states_visited.append(current_state) 
            ## Add the reward to states visited 
            for state in states_visited:
                n_visits[state] = n_visits[state] + 1.0
                G[state] = G[state] + (reward - G[state])/n_visits[state]
        ## Update the current state for next transition
        current_state = s_prime 
    return (G, n_visits) 

Why is the algorithm used in the `MC_generate_episode` considered first visit Monte Carlo?

- ANS: Its considered first visit because the reward of an state is only computed the first time the state is visited.

Now you have all the components in place to use the Monte Carlo algorithm to compute state values. You may use the `MC_state_values` function from the lesson notebook as a starting point. The main work done in this function is to iterate over the required number of episodes to compute the MC state value estimates. 

Execute your function for 5,000 episodes, to evaluate the initial uniform policy. 

In [9]:
def MC_state_values(policy, n_episodes):
    '''Function that evaluates the state value of 
    a policy using the Monte Carlo method.'''
    ## Create list of states 
    states = list(initial_policy.keys())
    n_states = len(states)
    
    ## An array to hold the accumulated returns as we visit states
    G = np.zeros((n_states))
    
    ## An array to keep track of how many times we visit each state so we can 
    ## compute the mean
    n_visits = np.zeros((n_states))
    
    ## Iterate over the episodes
    for i in range(n_episodes):
        G, n_visits = MC_episode(policy, G, n_visits, i, n_states) # neighbors, i, n_states)
    return(G)

nr.seed(123)
state_values = MC_state_values(initial_policy, n_episodes = 5000)
print(state_values.reshape((5, 5)))

[[-0.18415704 -0.20095358 -0.20906169 -0.20682251 -0.19227181]
 [-0.14370168  0.          0.          0.         -0.16200441]
 [-0.03373709  0.49458147  0.          0.46750852 -0.04607694]
 [-0.13636735  0.          0.          0.         -0.13166375]
 [-0.16253972 -0.18779938 -0.19566723 -0.18836134 -0.17501923]]


Make sure that the state values of the terminal state and taboo state must be zero. If they are not, there must be something wrong with your code.  

Examine your results. Are do the state values decrease with distance from the terminal state, and is this consistent with sampling the gain with the Monte Carlo algorithm? With 5,000 episodes sampled can you see evidence of the high variance inherent in MC methods? To answer the this second question it might help you to re-compute the state values using fewer or more episodes. 

- ANS 1 & 2: The value decreasing is consistent with the distance to the goal but is not symetrical which seems to be related to question number two. The high variance is evident at 5000 episodes which presents different values for each of the 4 branches (left-up, left-down, right-up, right-down).

## Monte Carlo Policy Improvement

Now you are ready to perform Monte Carlo policy improvement. As a first step you will need a function to compute the action values (Q values) for one episode. You may start with the `MC_action_value_episode` function from the lesson notebook. This function takes the following arguments, the current policy dictionary, the Q numpy array, the number of visits numpy array, the starting state, the number of states, and the number of actions. The function returns the updated Q numpy array and the number of visits numpy array. In the cell below create and test this function.     

In [10]:
import itertools

def MC_action_value_episode(policy, Q, n_visits, inital_state, n_states, n_actions, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    '''Function creates the Monte Carlo samples of action values for one episode.
    This function does most of the real work'''
    ## For each episode we use a list to keep track of states we have visited.
    ## Once we visit a state we need to accumulate values to get the returns
    state_actions_visited = np.zeros((n_states, n_actions))
    
    current_state = initial_state
    terminal = False
    taboo = False
    while(not terminal):
        ## Find the next action and reward
        action, s_prime, reward, terminal, taboo = take_action(current_state, policy)

        action_idx = action_index[action]         
        
        ## Check if this state-action has been visited.
        if(state_actions_visited[current_state, action_idx] != 1.0):
            ## Mark that the current state-action has been visited 
            state_actions_visited[current_state, action_idx] = 1.0  
            ## This is first vist MS, so must loop over all state-action pairs and 
            ## add the reward and increment the count for the ones visited.
            for s,a in list(itertools.product(range(n_states), range(n_actions))):
                ## Add reward to if these has been a visit to the state
                if(state_actions_visited[s,a] == 1.0):
                    n_visits[s,a] = n_visits[s,a] + 1.0
                    Q[s,a] = Q[s,a] + (reward - Q[s,a])/n_visits[s,a]    
        ## Update the current state for next transition
        current_state = s_prime
    return (Q, n_visits) 

## Basic test of the function
n_actions = 4
n_states = 25
Q = np.zeros((n_states, n_actions))
n_visits = np.zeros((n_states, n_actions))
initial_state = 24
Q, n_visits = MC_action_value_episode(initial_policy, Q, n_visits, initial_state, n_states, n_actions)
print('Q')
print(Q)
print('\nn_visits')
print(n_visits)

Q
[[-1.57142857e-01 -1.25925926e-01 -9.09090909e-02 -9.16666667e-02]
 [ 2.16666667e-01  3.27272727e-01 -9.13043478e-02 -4.76190476e-02]
 [ 4.60000000e-01 -4.50000000e-02  1.92307692e-01  5.26315789e-03]
 [ 6.22222222e-01  1.11111111e-02  1.71428571e-01  7.05882353e-02]
 [ 8.25000000e-01  1.08571429e+00  1.53333333e-01  8.12500000e-02]
 [-1.55172414e-01 -1.26923077e-01  4.50000000e+00 -1.28000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.97500000e+00  1.28333333e+00  2.66666667e+00  0.00000000e+00]
 [-1.53333333e-01 -1.51612903e-01  0.00000000e+00 -1.73529412e-01]
 [-1.75757576e-01  0.00000000e+00 -1.50000000e-01  1.00000000e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.56000000e+00 -2.61224490e-01  0.00000000e+00 -2.7600000

Next, in the cell below you need to create and test a function to compute the action value estimates over a number of episodes. You may start with the 'MC_action_values' function from the TD lesson notebook. The function has arguments of the initial policy, the action value matrix, the number of episodes to sample and the inital or starting state. The function returns the action value matrix. Execute your function for 10000 episodes, saving the results in a numpy array, and print the array.  

In [11]:
def print_Q(Q):
    Q = pd.DataFrame(Q, columns = ['up', 'down', 'left', 'right'])
    print(Q)

def MC_action_values(policy, Q, n_episodes, inital_state):
    '''Function evaluates the action-values given a policy for the specified number of episodes and 
    initial state'''
    n_states = len(policy)
    n_actions = len(policy[0])
    ## Array to count visits to action-value pairs
    n_visits = np.zeros((n_states, n_actions))
    ## Dictionary to hold neighbor states
    neighbors = {}
    
    ## Loop over number of episodes
    for _ in range(n_episodes):
        ## One episode of MC
        Q, n_visits = MC_action_value_episode(policy, Q, n_visits, initial_state, n_states, n_actions)
    return(Q)
    
## Basic test of the function
n_episodes = 10000
initial_state = 0
Q = np.zeros((n_states, n_actions))
initial_state = 0
Q = MC_action_values(initial_policy, Q, n_episodes, initial_state)
print_Q(Q)  

          up      down       left      right
0  -0.128441 -0.083002  -0.128303  -0.147070
1  -0.117423 -0.114696  -0.084558  -0.130547
2  -0.096034 -0.097584  -0.078226  -0.099943
3  -0.070123 -0.069783  -0.056112  -0.056477
4  -0.024947  0.014660  -0.023224  -0.019677
5  -0.085925  0.023483  -0.056376  -0.056209
6   0.000000  0.000000   0.000000   0.000000
7   0.000000  0.000000   0.000000   0.000000
8   0.000000  0.000000   0.000000   0.000000
9   0.032556  0.141982   0.049170   0.047570
10 -0.002273 -0.031653   0.057252   0.521086
11  0.549162  0.568746   0.137219  10.000000
12  0.000000  0.000000   0.000000   0.000000
13  0.825565  0.800223  10.000000   0.315862
14  0.123702  0.064157   0.731229   0.192476
15  0.143884 -0.058311   0.013339   0.017325
16  0.000000  0.000000   0.000000   0.000000
17  0.000000  0.000000   0.000000   0.000000
18  0.000000  0.000000   0.000000   0.000000
19  0.258298  0.034004   0.117469   0.101390
20  0.064469 -0.013976  -0.015206  -0.055278
21 -0.0209

Make sure that the taboo states and the terminal state all have zero action values for every action. If not, there is a bug in your code. 

Find the largest action values in this array. Are these values consistent with the reward structure of this problems and why?

- ANS: The highest action values in the array are 11 right and 13 left, which are consistent with the rewards because they transition into the terminal state  

With the action value array computed, in the cell below you can create and test code to find an improved policy. You may use the `update_policy` function from the TD lesson notebook. The function has arguments of an initial policy, an action value array, the $\epsilon$-greedy parameter, and returns an improved policy. 

Before executing your `update_policy` function will need to create a deep copy of your initial dictionary. Do this with the [copy.deepcopy function](https://docs.python.org/3/library/copy.html).  

Execute your code with $\epsilon = 0.01$, and print the policy computed.

In [12]:
def update_policy(policy, Q, epsilon, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    '''Updates the policy based on estiamtes of Q using 
    an epslion greedy algorithm. The action with the highest
    action value is used.'''
    
    ## Find the keys for the actions in the policy
    keys = list(policy[0].keys())
    
    ## Iterate over the states and find the maximm action value.
    for state in range(len(policy)):
        ## First find the index of the max Q values  
        q = Q[state,:]
        max_action_index = np.where(q == max(q))[0]

        ## Find the probabilities for the transitions
        n_transitions = float(len(q))
        n_max_transitions = float(len(max_action_index))
        p_max_transitions = (1.0 - epsilon *(n_transitions - n_max_transitions))/(n_max_transitions)
  
        ## Now assign the probabilities to the policy as epsilon greedy.
        for key in keys:
            if(action_index[key] in max_action_index): policy[state][key] = p_max_transitions
            else: policy[state][key] = epsilon
    return(policy)


updated_policy = deepcopy(initial_policy)
updated_policy = update_policy(updated_policy, Q, 0.01)
updated_policy

{0: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 1: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 2: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 3: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 4: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 5: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 6: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 7: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 8: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 9: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 10: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 11: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 12: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 13: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 14: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 15: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 16: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 17: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 18: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 19: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r

Examine your impoved policy. Are the most probable actions consitent with an optimal policy and why? 

- ANS: Most of them are consistent with the optimal solution with exception of state 3 which incorrectly decides to go left instead of right, adding an unnecessary step. Another interesting note is that none of the actions have a 0.0 probability due the epsilon hyper parameter.

Now, compute and print the Monte Carlo state values for the improved policy in the cell below. Use 10,000 episodes to compute these state value estimates. 

In [13]:
nr.seed(123)
state_values = MC_state_values(updated_policy, n_episodes = 10000)
print(state_values.reshape((5,5)))

[[2.32652065 1.84600322 1.5208282  1.29407064 2.31524113]
 [3.1445079  0.         0.         0.         3.17436286]
 [4.75080507 9.66764319 0.         9.64628078 4.75051043]
 [3.16261451 0.         0.         0.         3.13912959]
 [2.33545013 1.84215111 1.50974564 1.8070079  2.30885796]]


Compare these state values to the MC state values you computed using the initial random policy. Is there a clear improvement across all states?    

- ANS: It does not seem to be a lot of improvement, the results are still unbalanced. The most notable change is a more uniform distribution of the state values in a nice gradient.

## TD(0) Policy Evaluation

With the analysis using Monte Carlo RL algorithms completed, you will perform the same analysis using the TD(0)/SARSA(0) methods. You will then be able to compare the results using these two methods.  

As a first step you will need a function to compute state values using the TD(0) algorithm. You are welcome to start with the `td_0_state_values` function from the TD learning notebook. The arguments to this function are policy the number of episodes, alpha (the learning rate) and gamma (discount factor). This function uses the `take_action` function you defined previously. Keep in mind that you may need to modify this code to correctly treat the taboo states. Specifically, taboo states should not be visited. 

Execute your code for the initial policy to test it using 20,000 episodes, a learning rate of 0.01 and a discount rate of 1.0.  

In [14]:
def td_0_state_values(policy, n_samps, alpha = 0.01, gamma = 1.0):
    """
    Function for TD(0) policy evalutation
    """
    
    ## Find the starting state
    n_states = len(policy)
    current_state = start_episode(n_states)
    terminal = False
    
    ## Array for state values
    v = np.zeros((n_states,1))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action, s_prime, reward, terminal, taboo = take_action(current_state, policy)
        ## Compute the TD error
        delta = reward + gamma*v[s_prime] - v[current_state]
        ## Update the state value
        v[current_state] = v[current_state] + alpha*delta
        current_state = s_prime
        if(terminal): ## start new episode when terminal
            current_state = start_episode(n_states)
    return(v)

td_0_state_values(updated_policy, 20000).reshape((5,5))        

array([[9.62375986, 9.48552356, 8.91521003, 6.12236861, 8.28169025],
       [9.73420671, 0.        , 0.        , 0.        , 9.64935481],
       [9.85023895, 9.95812201, 0.        , 9.98304511, 9.86652201],
       [9.73633315, 0.        , 0.        , 0.        , 9.70681935],
       [9.58348076, 9.18592363, 6.92386937, 7.35523842, 9.35425234]])

Make sure that the state values of the terminal state and taboo state must be zero. If they are not, there must be something wrong with your code.  

Examine your results. Do these state values seem consistent with what you would expect for this problem, and why? Compare these results to the unbiased results of the MC state value estimates. Can you see evidence of the bias of the TD(0) algorithm, and if so, what is it? To explore the answer to the second question you can try different values of the learning rate, alpha and see how this affects the bias. If you use small values of alpha, you will need to increase the number of episodes sampled.

- ANS 1: Because TD(0) is doing estimates of estimates instead of looking at the entire episode, there is a bias towards the most recent previous state.
- ANS 2: I believe there es evidence of this bias on the gradient of the state values, on MC there is more of a gradient while in TD(0) the value difference between states is more vanishing.


## SARSA(0) Policy Improvement

Now you will perform policy improvement using the SARSA(0) algorithm.  You are welcome to start with the `new_episode` and `SARSA_0` functions from the TD/Q-learning notebooks.    

The `new_episode` function starts an episode to sample action values. This function has arguments of number of states and the policy. The function calls the `start_episode` and `take_action` functions you created previously. The function returns the starting state of the episode, the action, the successor state, the reward, and a flag indicating if the successor state is terminal. 

The `SARSA_0` function estimates action value (Q) over a specified number of episodes. The function has arguments of policy, number of episodes sampled, the learning rate (alpha) and the discount rate (gamma). The function uses the `new episode` and `take_action` functions. The state and successor state information is updated at each time step. An action value array is returned.  

Execute your code for 20,000 episodes, and with $\alpha = 0.05$, and $\gamma = 1.0$.

In [15]:
def new_episode(n_states, policy):
    '''This function provides a start for a TD
    episode making sure the first transition is not 
    the termnal state'''
    current_state = start_episode(n_states)
    ## Find fist action and reward
    action, s_prime, reward, terminal, taboo = take_action(current_state, policy)
    return(current_state, action, s_prime, reward, terminal, taboo)    


def SARSA_0(policy, n_samps, alpha = 0.05, gamma = 1.0, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    """
    Function for TD(0) policy evalutation
    """
    
    ## Find the starting state
    n_states = len(policy)
    current_state, action, s_prime, reward, terminal, taboo = new_episode(n_states, policy)
    action_idx = action_index[action]
    
    ## Array for state values
    q = np.zeros((n_states, len(policy[0])))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action_prime, s_prime_prime, reward_prime, terminal_prime, taboo_prime = take_action(s_prime, policy)
        action_idx_prime = action_index[action_prime]
        ## Compute the TD error
        delta = reward + gamma*q[s_prime, action_idx_prime] - q[current_state, action_idx]
        ## Update the action values
        q[current_state, action_idx] = q[current_state, action_idx] + alpha*delta
        ## Update the state, action and reward for the next time step
        current_state = s_prime
        s_prime = s_prime_prime
        action = action_prime
        reward = reward_prime
        terminal = terminal_prime
        action_idx = action_idx_prime

        ## Check if end of episode
        if(terminal): 
            ## start new episode
            current_state, action, s_prime, reward, terminal, taboo = new_episode(n_states, policy)        
    return(q)


Q = SARSA_0(initial_policy, 20000, alpha = 0.05, gamma = 1.0)
print_Q(Q)

          up      down      left     right
0  -7.896581 -5.926941 -7.457062 -7.400567
1  -8.257931 -7.950987 -6.873939 -7.499640
2  -8.611263 -8.615417 -7.342902 -7.618114
3  -8.535842 -8.879594 -7.844612 -6.969505
4  -8.047034 -5.924891 -7.567655 -7.838854
5  -6.745346 -4.042400 -6.983533 -7.134553
6   0.000000  0.000000  0.000000  0.000000
7   0.000000  0.000000  0.000000  0.000000
8   0.000000  0.000000  0.000000  0.000000
9  -6.922935 -5.165865 -6.919614 -6.998032
10 -6.030415 -5.844126 -5.408078 -2.097826
11 -2.620769 -2.386957 -3.727376  1.005919
12  0.000000  0.000000  0.000000  0.000000
13 -2.962500 -3.394661 -0.350175 -4.142490
14 -6.146328 -6.089841 -2.911117 -5.923719
15 -4.366163 -7.456284 -7.273841 -6.933417
16  0.000000  0.000000  0.000000  0.000000
17  0.000000  0.000000  0.000000  0.000000
18  0.000000  0.000000  0.000000  0.000000
19 -5.180597 -7.127585 -6.769803 -6.636247
20 -5.861091 -8.525401 -8.365185 -8.358296
21 -9.315637 -9.125317 -7.560640 -8.473807
22 -9.45690

Examine the action values you have computed. Ensure that the action values are 0 for the goal and taboo states. Also check that the actions with the largest values for each state make sense in terms of reaching the goal. 

With the action values computed, you will now find an improved policy. Make sure you create a copy of the initial random policy with `copy.deepcopy`. Execute the `update_policy` function using the copy of the initial random policy, the action value array you computed, and with $\epsilon = 0.01$.

In [16]:
sarsa_policy = deepcopy(initial_policy)
sarsa_policy = update_policy(sarsa_policy, Q, 0.01)
sarsa_policy

{0: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 1: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 2: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 3: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 4: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 5: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 6: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 7: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 8: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 9: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 10: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 11: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 12: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 13: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 14: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 15: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 16: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 17: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 18: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 19: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r

Examine the policy you have computed. Make sure that the most probable actions lead to an optimal path to the terminal state from any other state, that is not taboo. 

Now, you will evaluate your policy using both the MC and TD(0) algorithms. In the cells below execute the required functions and display the results. For the `MC_state_values` function use 10,000 episodes.  For the `td_0_state_values` function use 20,000 episodes, a learning rate of 0.01 and a discount rate of 1.0. 

In [17]:
state_values = MC_state_values(sarsa_policy, n_episodes = 10000)
print(state_values.reshape((5,5)))

[[2.33536532 1.84309587 1.52763238 1.81900038 2.30137082]
 [3.15835498 0.         0.         0.         3.13367644]
 [4.78287882 9.73254801 0.         9.61809754 4.72494417]
 [3.18222093 0.         0.         0.         3.14032063]
 [2.35292605 1.86409819 1.53282857 1.8497434  2.33064935]]


In [18]:
td_state_values = td_0_state_values(sarsa_policy, 20000)
print(td_state_values.reshape((5,5)))

[[9.57948434 9.0703385  6.76813062 7.22851585 9.40790773]
 [9.74117687 0.         0.         0.         9.75412913]
 [9.87284797 9.98109748 0.         9.97097747 9.876213  ]
 [9.74862021 0.         0.         0.         9.75049226]
 [9.43844633 7.62679961 7.2281707  9.23206852 9.61726625]]


Compare the state values computed by the two algorithms. Do both methods compute state-values consistent with the problem, and why? Why do you think these estimates of state value are different? 

- ANS 1: Both compute similar state-values that are consistent with the problem but they are not equal to each other.
- ANS 2: I think is related to the TD(0) greedy bias, while MC shows a more pronounced gradient the short memory of TD(0) returns a vanishing gradient.

## Apply Double Q-Learning

Finally, you will apply Double Q-learning(0) to the warehouse navigation problem. 

### Q-Learning Environment Simulator

Since Q-learning is different from MC or TD/SARSA learning, you will need to create a new set of environment simulator functions. Keep in mind that the simulator is not part of the agent and the agent can only interact with the environment through the simulations. 

You may use the Q-learning simulator code from the Q-learning notebook as a starting point to create the following functions. It will help to change the names of these functions, by say, prefixing Q_ to prevent name space conflicts with previously defined functions. 


**Q_action_lookup** and **Q_index_lookup** functions to translate between an index and the corresponding action or the action and the corresponding index.  

**Q_next_state** function returns the next state given arguments of state and action. 

**Q_simulate_environment** function is similar to the `simulate_environment` function you have been working with. The arguments are state and action. The function then returns the successor or state, the an array of rewards from the successor state, and a flag to indicate if the successor state is terminal. 

In [19]:
def q_action_lookup(index):
    """Helper function returns action given an index"""
    action_dic = {0:'u', 1:'d', 2:'l', 3:'r'}
    return action_dic[index]

def q_index_lookup(action):
    """Helper function returns index given action"""
    index_dic = {'u':0, 'd':1, 'l':2, 'r':3}
    return index_dic[action]


def q_next_state(state, action_index, neighbors = neighbors, action_lookup = q_action_lookup):
    return(neighbors[state][q_action_lookup[action_index]])

def q_simulate_environment(s, action, neighbors = neighbors, rewards = rewards):
    """
    Function simulates the environment for Q-learning.
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward_prime = np.array([rewards[s_prime][a] for a in rewards[0].keys()])
    return (s_prime, reward_prime, is_terminal(s_prime))

for a in ['u', 'd', 'r', 'l']:
    print(q_simulate_environment(1, a))

(1, array([-1. , -1. , -0.1, -0.1]), False)
(1, array([-1. , -1. , -0.1, -0.1]), False)
(2, array([-1. , -1. , -0.1, -0.1]), False)
(0, array([-1. , -0.1, -1. , -0.1]), False)


### Random Episode Start

In the cell below, create and test a function to find a random start for each episode. You can define the `Q_start_episode` function starting with the function in the MC RL lesson notebook. The function takes arguments of number of states and number of actions. The function uses the `is_terminal` and `Q_simulate_environment` functions you defined earlier. The function returns the state, initial action index and the initial reward. 
    

In [20]:
def q_start_episode(n_states, n_actions):
    '''Function to find a random starting values for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state)):  ## Make sure not starting at the terminal state
         state = nr.choice(range(n_states))
    ## Now find a random starting action index
    a_index = nr.choice(range(4), size = 1)[0]
    s_prime, reward, terminal = q_simulate_environment(state, q_action_lookup(a_index))   
    return state, a_index, reward[a_index]

[q_start_episode(25,4) for _ in range(10)]

[(18, 0, -1.0),
 (19, 1, -1.0),
 (20, 1, -1.0),
 (20, 2, -1.0),
 (14, 1, -0.1),
 (18, 3, -1.0),
 (10, 3, 10.0),
 (11, 2, -1.0),
 (22, 0, -1.0),
 (23, 0, -1.0)]

### Double Q-Learning    

It is now time to pull together the functions to implement the double Q-learning algorithm. There are two functions, which you can use the functions from the MC lesson notebook as a starting point.   

**update_double_Q** is a function that computes the error, using the double Q update, and updates the Q value. The function takes the arguments of the two Q arrays, the current state, the action index, the current reward, the learning rate (alpha) and the discount rate (gamma). The function uses the `Q_simulate_environment` function. The function returns the updated Q array, the successor state, the successor reward, a flag indicating if the successor state is terminal and the index of the successor action.   

**double_Q_learning_0** is a function that randomly updates one or the other Q arrays using the Q array as an argument over the specified number of episodes. The function has arguments of policy, number of episodes, the learning rate (alpha), and the discount rate (gamma). The function makes calls to the `update_double_Q` function. An updated Q array is returned.   

Execute this code using the initial random policy, 10,000 episodes, a learning rate of 0.01 and a discount rate of 1.0.   

In [21]:
def update_double_Q(q1, q2, current_state, a_index, reward, alpha, gamma):
    """Function to update the actions values in the Q matrix"""
    ## Get s_prime given s and a
    s_prime, reward_prime, terminal = q_simulate_environment(current_state, q_action_lookup(a_index))
    a_prime_index = nr.choice(np.where(reward_prime == max(reward_prime))[0], size = 1)[0]
    ## Update the action values 
    q1[current_state,a_index] = q1[current_state,a_index] + alpha * (reward + gamma * (q2[s_prime,a_prime_index] - q1[current_state,a_index]))
    return q1, s_prime, reward_prime, terminal, a_prime_index

def double_Q_learning_0(policy, episodes, alpha = 0.02, gamma = 0.9):
    """
    Function to perform Q-learning(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    
    ## Initialize both Q matricies
    Q1 = np.zeros((n_states,n_actions))
    Q2 = np.zeros((n_states,n_actions))
    
    for _ in range(episodes): # Loop over the episodes
        terminal = False
        ## Find the inital state, action index and reward
        current_state, a_index, reward = q_start_episode(n_states,n_actions)
        
        while(not terminal): # Episode ends where get to terminal state   
            ## Update the action values in Q1 or Q2 based on random choice
            if(nr.uniform() <= 0.5):
                Q1, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q1, Q2, current_state, a_index, reward, alpha, gamma)
            else:
                Q2, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q2, Q1, current_state, a_index, reward, alpha, gamma)
            ## Set action, reward and state for next iteration
            a_index = a_prime_index
            current_state = s_prime
            reward = reward_prime[a_prime_index]
    return(Q1)

Q = double_Q_learning_0(initial_policy, 10000, 0.01, 1.0)
print_Q(Q)

          up      down      left     right
0   0.781624  5.321024  0.691548  4.087700
1   0.549922  0.603091  4.578646  3.814815
2   0.138366  0.132464  4.073462  3.932899
3   0.479787  0.378653  3.821795  4.474893
4   0.673250  5.389517  3.951603  0.594394
5   4.658040  6.884919  1.245880  1.089167
6   0.716531  4.551099  1.742243  0.066297
7   0.841578  0.000000  0.506498  0.543123
8   0.863008  4.351946  0.074697  1.936121
9   4.535472  6.628181  1.137764  1.171594
10  5.427298  5.458656  1.219607  9.890603
11  3.380822  2.960315  2.008630  9.764553
12  0.000000  0.000000  0.000000  0.000000
13  3.693759  3.545133  9.629521  2.020484
14  5.352718  5.392446  9.778753  2.157142
15  6.531127  4.467529  1.371583  0.965954
16  4.363842  0.476190  1.857633  0.095655
17  0.000000  0.815840  0.377750  0.555327
18  4.913699  0.736250  0.096905  1.611348
19  6.633234  4.445445  1.375585  1.004130
20  5.581434  0.578332  0.703895  3.973622
21  0.502973  0.504307  4.508556  3.797044
22  0.31423

Make sure that the action values of the taboo states and the terminal state are  all 0.

With the action values computed using the action values, you will now find an improved policy. Make sure you create a copy of the initial random policy with `copy.deepcopy`. Execute the `update_policy` function using the copy of the initial random policy, the action value array you computed, and with $\epsilon = 0.01$.

In [22]:
double_q_policy = deepcopy(initial_policy)
double_q_policy = update_policy(double_q_policy, Q, 0.01)
double_q_policy

{0: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 1: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 2: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 3: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 4: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 5: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 6: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 7: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 8: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 9: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 10: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 11: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.97},
 12: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25},
 13: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 14: {'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01},
 15: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 16: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 17: {'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01},
 18: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01},
 19: {'u': 0.97, 'd': 0.01, 'l': 0.01, 'r

Now, you will evaluate your policy using the MC algorithm. In the cells below execute the required function with 10,000 episodes and display the results. 

In [23]:
double_q_state_values = MC_state_values(double_q_policy, n_episodes = 10000)
print(double_q_state_values.reshape((5,5)))

[[2.33674289 1.84793987 1.50314319 1.83198847 2.32336832]
 [3.1592799  0.         0.         0.         3.15586883]
 [4.76635504 9.67537909 0.         9.67735359 4.75293853]
 [3.15780718 0.         0.         0.         3.13429044]
 [2.3404878  1.8540889  1.5192926  1.83980011 2.31559689]]


Are these state values reasonably similar to those found with the SARSA and is this what you expect and why? 

- ANS 1: Yes, this results are consistent with SARSA. I did expect this because SARSA and Q-Learning are very similar algorithms, while Q will learn the optimal policy, SARSA will do learn the near optimal policy because it allows more exploration.